# Locally abstract type

##### 🟢 &nbsp; &nbsp; Basic

In [1]:
let wrap_list = fun (type a) x -> [x] ;;
wrap_list 10 ;;
wrap_list 10.10 ;;

let wrap_tuple (type a b) a b = (a, b) ;;
wrap_tuple 10 10 ;;
wrap_tuple 1.11 2.22 ;;

val wrap_list : 'a -> 'a list = <fun>


- : int list = [10]


- : float list = [10.1]


val wrap_tuple : 'a -> 'b -> 'a * 'b = <fun>


- : int * int = (10, 10)


- : float * float = (1.11, 2.22)


<br/>

##### 🟢 &nbsp; &nbsp; Samples

In [16]:
let count_by (type k) (lst: 'a list) ~compare:cmp ~(projection: 'a -> k) =
    let len = List.length lst in
    if len = 0 then []
    else
        let module M = Map.Make(struct
            type t = k
            let compare = cmp
        end) in
        let rec loop l acc =
            match l with
            | [] -> acc
            | h :: t -> 
                let key = projection h in
                loop t (acc |> M.update key (function None -> Some (1) | Some x -> Some (x + 1)))
        in
        loop lst M.empty |> M.bindings ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> count_by ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> count_by ~compare:Float.compare ~projection:(fun x -> x) ;;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] in
people |> count_by ~compare:String.compare ~projection:(fun x -> x.name) ;;

val count_by :
  'a list ->
  compare:('k -> 'k -> int) -> projection:('a -> 'k) -> ('k * int) list =
  <fun>


- : (int * int) list = [(1, 2); (2, 3); (3, 2); (4, 2)]


- : (Float.t * int) list = [(1., 2); (2., 3); (3., 2); (4., 2)]


type person = { name : string; age : float; }


- : (String.t * int) list = [("Click", 3); ("Jack", 2)]


In [6]:
let distinct (type a) (lst:a list) ~(compare:a -> a -> int) = 
    let module S = Set.Make(struct 
        type t = a
        let compare = compare
    end) in
    lst |> S.of_list |> S.elements ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> distinct ~compare:Int.compare ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> distinct ~compare:Float.compare ;;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] 
and cmp x y = 
    match (String.compare (String.lowercase_ascii x.name) (String.lowercase_ascii y.name), Float.compare x.age y.age) with
    | (0,0) -> 0
    | (a,b) -> if a > b then a else b in    
people |> distinct ~compare:cmp ;;

val distinct : 'a list -> compare:('a -> 'a -> int) -> 'a list = <fun>


- : int list = [1; 2; 3; 4]


- : Float.t list = [1.; 2.; 3.; 4.]


type person = { name : string; age : float; }


- : person list =
[{name = "Click"; age = 13.}; {name = "Jack"; age = 13.};
 {name = "Jack"; age = 14.}; {name = "Click"; age = 31.}]


In [11]:
let distinct_by (type k) (lst:'a list) ~(compare:k -> k -> int) ~(projection:'a -> k) =
    let module S = Set.Make(struct 
        type t = k
        let compare = compare
    end) in
    let rec loop l keys ret =
        match l with
        | [] -> ret
        | h :: t ->
            let key = projection h in
            if keys |> S.exists (fun x -> x = key) then loop t keys ret
            else loop t (keys |> S.add key) (h :: ret)
    in
    loop lst S.empty [] ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> distinct_by ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> distinct_by ~compare:Float.compare ~projection:(fun x -> x);;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] ;;
people |> distinct_by ~compare:String.compare ~projection:(fun x -> x.name) ;;
people |> distinct_by ~compare:Float.compare ~projection:(fun x -> x.age) ;;

val distinct_by :
  'a list -> compare:('k -> 'k -> int) -> projection:('a -> 'k) -> 'a list =
  <fun>


- : int list = [4; 3; 2; 1]


- : Float.t list = [4.; 3.; 2.; 1.]


type person = { name : string; age : float; }


val people : person list =
  [{name = "Click"; age = 13.}; {name = "Click"; age = 12.};
   {name = "Jack"; age = 13.}; {name = "Jack"; age = 14.};
   {name = "Click"; age = 31.}]


- : person list = [{name = "Jack"; age = 13.}; {name = "Click"; age = 13.}]


- : person list =
[{name = "Click"; age = 31.}; {name = "Jack"; age = 14.};
 {name = "Click"; age = 12.}; {name = "Click"; age = 13.}]


In [14]:
let group_by (type k) (lst:'a list) ~(compare:k -> k -> int) ~(projection:'a -> k) =
    let module M = Map.Make(struct
        type t = k
        let compare = compare
    end) in
    let rec loop l ret =
        match l with
        | [] -> ret
        | h :: t ->
            let key = projection h in
            loop t (ret |> M.update key (function Some x -> Some (h :: x) | _ -> Some [h]))
    in
    loop lst M.empty |> M.bindings ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> group_by ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> group_by ~compare:Float.compare ~projection:(fun x -> x);;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] ;;
people |> group_by ~compare:String.compare ~projection:(fun x -> x.name) ;;
people |> group_by ~compare:Float.compare ~projection:(fun x -> x.age) ;;

val group_by :
  'a list ->
  compare:('k -> 'k -> int) -> projection:('a -> 'k) -> ('k * 'a list) list =
  <fun>


- : (int * int list) list =
[(1, [1; 1]); (2, [2; 2; 2]); (3, [3; 3]); (4, [4; 4])]


- : (Float.t * Float.t list) list =
[(1., [1.; 1.]); (2., [2.; 2.; 2.]); (3., [3.; 3.]); (4., [4.; 4.])]


type person = { name : string; age : float; }


val people : person list =
  [{name = "Click"; age = 13.}; {name = "Click"; age = 12.};
   {name = "Jack"; age = 13.}; {name = "Jack"; age = 14.};
   {name = "Click"; age = 31.}]


- : (String.t * person list) list =
[("Click",
  [{name = "Click"; age = 31.}; {name = "Click"; age = 12.};
   {name = "Click"; age = 13.}]);
 ("Jack", [{name = "Jack"; age = 14.}; {name = "Jack"; age = 13.}])]


- : (Float.t * person list) list =
[(12., [{name = "Click"; age = 12.}]);
 (13., [{name = "Jack"; age = 13.}; {name = "Click"; age = 13.}]);
 (14., [{name = "Jack"; age = 14.}]); (31., [{name = "Click"; age = 31.}])]
